In [1]:
import pandas as pd #Importar dataset
import math
import random

In [2]:
def dividir_datos(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio) #Tamaño de datos de entrenamiento
    trainSet = []
    test = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(test)) #Tomamos datos al azar
        trainSet.append(test.pop(index))  #Agregamos datos al vector 
    return [trainSet, test]               #Tenemos los datos de entrenamiento y test

In [3]:
def separar_clases(dataset):   #Separar los datos por clase
    separar = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separar):
            separar[vector[-1]] = []
        separar[vector[-1]].append(vector)
    return separar

In [4]:
def media(numbers):    #Función para sacar la media
    return sum(numbers)/float(len(numbers))

In [5]:
def stdev(numbers):  #Función para sacar la variación
    avg = media(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1) 
    return math.sqrt(variance)

In [6]:
def sumarizar(dataset): #Función para sumarizar los datos por clase
    summaries = [(media(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [7]:
def summarizeByClass(dataset): #Sumarizar los datos por clase (Juntar los datos por clase)
    separar = separar_clases(dataset)
    summaries = {}
    for classValue, instances in separar.items():
        summaries[classValue] = sumarizar(instances)
    return summaries

In [8]:
def calculateProbability(x, media, stdev): #Fórmula probabilidad Bayesiana
    exponent = math.exp(-(math.pow(x-media,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [9]:
def calculateClassProbabilities(summaries, inputVector): #Calculamos la probabilidad de cada clase
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            media, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, media, stdev) #Llamamos la función del cálculo de la probabilidad
    return probabilities

In [10]:
def predict(summaries, inputVector):  #FUnción para calcular la predicción
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [11]:
def ob_predicciones(summaries, testSet): #calculamos la predicción
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])  #Llamamos a la función de predicción
        predictions.append(result)
    return predictions

In [12]:
def tasa_pred(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100

In [14]:
df = pd.read_csv("dataset.csv")        #Importar dataset
dataset = df.astype(float).values.tolist()  #Convertimos todo a float
splitRatio = 0.67                 #Que tanto del dataset se divide en entrenemiento y test
trainingSet, testSet = dividir_datos(dataset, splitRatio)     #Se llama a la función para dividir en entrenamiento y test
print('Datos=', len(dataset),',Datos de entrenamiento=',len(trainingSet),',Datos de test=',len(testSet))   #Imprimir el número de datos y cuantos son de entrenamiento y test
summaries = summarizeByClass(trainingSet)   # Sacamos media aritmetica de cada clase
predictions = ob_predicciones(summaries, testSet) #Hacemos la predicción
accuracy = tasa_pred(testSet, predictions)     # Calculamos el porcentaje de predicción
print('Accuracy:',accuracy)                 # Imprimimos el porcentaje de predicción

Datos= 390 ,Datos de entrenamiento= 261 ,Datos de test= 129
Accuracy: 36.434108527131784
